# ESG Data Preprocessing

In [1]:
### package
import os 
import pandas as pd
import numpy as np
import copy
os.chdir("C:/Users/user/Desktop/ESG데이터/ESG")

In [2]:
### Data

## ESG데이터
raw_esg = pd.read_excel("ESG_DATA.xlsx",converters={'종목코드':str})

## Environmental
E = pd.read_excel("E_final.xlsx",converters={'종목코드':str})

## Social
S = pd.read_excel("S_final.xlsx",converters={'종목코드':str})

## Governance
G = pd.read_excel("G_final.xlsx",converters={'종목코드':str})

In [3]:
### 분석대상 key 변수 : 분석대상에서 SBS미디어홀딩스, 에스케이머티리얼즈 제외
raw_esg = raw_esg[-raw_esg['종목코드'].isin(['101060','036490'])]
esg_data = raw_esg[['종목코드','회사명','SICS_Sector_Kr','ESG_GRADE']]

### ESG Total Key Risk

In [4]:
esg_tkr = pd.merge(left = esg_data, right = E[['종목코드','E_risk']], how = 'left', on = '종목코드')
esg_tkr = pd.merge(left = esg_tkr, right = S[['종목코드','S_risk']], how = 'left', on = '종목코드')
esg_tkr = pd.merge(left = esg_tkr, right = G[['종목코드','G_risk']], how = 'left', on = '종목코드')

In [5]:
### weight matrix
sics = pd.DataFrame({'소비재':[10/32,13/32,9/32],
                     '추출물 및 광물 처리':[22/43,11/43,10/43],
                     '금융':[6/30,9/30,15/30],
                     '식음료':[20/42,13/42,9/42],
                     '헬스케어':[12/38,17/38,9/38],
                     '인프라':[15/39,12/39,12/39],
                     '재생가능 자원 및 대체 에너지':[19/38,9/38,10/38],
                     '자원 변환':[19/43,14/43,10/43],
                     '서비스':[10/34,15/34,9/34],
                     '기술 및 통신':[16/48,20/48,12/48],
                     '운송':[16/37,12/37,9/37]})

In [6]:
a = [[0 for col in range(12)] for row in range(esg_tkr.shape[0])]
for i in range(esg_tkr.shape[0]) :
    a[i] = sics[esg_tkr['SICS_Sector_Kr'][i]].tolist()

esg_tkr['ESG_risk'] = np.diag(np.array(esg_tkr.iloc[:,4:]) @ np.array(a).T)

### ESG Data Set

In [8]:
esg_data = pd.merge(left = esg_tkr[['종목코드','SICS_Sector_Kr','ESG_GRADE','ESG_risk']], 
                    right = E[['종목코드','지가경공시','지가경발간','ESG임원비율','ESG채권','환경이사']], how = 'left', on = '종목코드')
esg_data = pd.merge(left = esg_data, right = S[['종목코드','여성임원비율']], how = 'left', on = '종목코드')
esg_data = esg_data.fillna(0)

ESG_GROUP = [0 for col in range(esg_data.shape[0])]
for i in range(esg_data.shape[0]) :
    if esg_data['ESG_GRADE'].astype(int)[i] in [1,2,3] :
        ESG_GROUP[i] = "G1"
    elif esg_data['ESG_GRADE'].astype(int)[i] in [4,5,6] :
        ESG_GROUP[i] = "G2"
    elif esg_data['ESG_GRADE'].astype(int)[i] in [7,8,9] : 
        ESG_GROUP[i] = "G3"
    else : ESG_GROUP[i] = "NA"
        
esg_data['Stratify'] = esg_data['SICS_Sector_Kr'] + "_" + ESG_GROUP
esg_data.to_excel("C:/Users/user/Desktop/ESG데이터/ESG_final.xlsx",index = False)
esg_data.to_excel("C:/Users/user/Desktop/ESG데이터/ESGI/ESG_final.xlsx",index = False)
esg_data.to_excel("C:/Users/user/Desktop/ESG데이터/Tableau/ESG_final.xlsx",index = False)